In [1]:
import pandas as pd
df = pd.read_csv('../../data/raw.csv')
#df.shape

features_baseline = ['is_female', 'age', 'height', 'weight', 'optime']
features_text = ['diagnoses', 'operations']
X = df[features_baseline + features_text]
y = df['aki_severity']

In [2]:
from TabuLLM.embed import TextColumnTransformer
trans_embed = TextColumnTransformer(
    model_type = 'st'
)
X_embedding = trans_embed.fit_transform(df[features_text])
X_2 = pd.concat([X_embedding, df[features_baseline]], axis=1)

c:\Users\alire\anaconda3\envs\devTEFE\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\alire\anaconda3\envs\devTEFE\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from TabuLLM.cluster import SphericalKMeans
from sklearn.preprocessing import TargetEncoder, StandardScaler

pipeline_text = Pipeline([
    ('cluster', SphericalKMeans(n_clusters=10, n_init=5, return_hard_labels=True)),
    ('te', TargetEncoder(smooth = 'auto'))
])
ct = ColumnTransformer([
    ('baseline', StandardScaler(), features_baseline),
    ('text', pipeline_text, X_2.columns)
], remainder = 'drop')
pipeline_final = Pipeline([
    ('preprocess', ct)
    , ('logit', LogisticRegression())
])

In [8]:
pipeline_final.fit(X_2, y)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('baseline', StandardScaler(),
                                                  ['is_female', 'age', 'height',
                                                   'weight', 'optime']),
                                                 ('text',
                                                  Pipeline(steps=[('cluster',
                                                                   SphericalKMeans(n_clusters=10,
                                                                                   n_init=5,
                                                                                   return_hard_labels=True)),
                                                                  ('te',
                                                                   TargetEncoder())]),
                                                  Index(['X_0', 'X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 'X_7', 'X_8', 'X_9',
       ...
       'X_379', 'X_380', 'X_381', 'X_382', 'X_383', 'is_female', 'age',
       'height', 'weight', 'optime'],
      dtype='object', length=389))])),
                ('logit', LogisticRegression())])

In [12]:
SphericalKMeans(n_clusters=10, n_init=5, return_hard_labels=True).fit_transform(X_embedding).shape

(830, 1)